In [1]:
# [x] see if we have multi query capabilities
# [x] expose connection radius infrastructure
# what we want:
# [x] we want to create new states first with a specific connection radius - implement a ConnectionFilter instead of trying to import BoundedConnectionStrategy
# [x] then we want to reconfigure the planner to only create new states when adding start and goals without growing the roadmap more (just 'using') - clear input query every time
# [x] decide on experiment apparatus
# Run from d: 2....20
# Naive: Pre-constructed instance (empty, narrow corridor, etc.). Then run with m (~500>) different roadmaps. Each roadmap is a single Monte-Carlo sample.
# Naive: Run n trials on one roadmap <-- problem: number of trials we need to run scales alongside epsilon nets. ironic! ~either sample new states or just reuse graph states? sample lots of states and reuse?
# [x] write up experiment apparatus
# [x] switch to manually defining problems instead of simple setup since we need more control
# [x] define validation of ONE d-dimensional roadmap
# [x] add in multiple d-dimensional roadmaps
# [x] add in control of graph growth in solution criterion
# [x] add in parameter/n prob solution infrastructure
# AS SAMPLES INCREASE m inc.
# maintain a list of tolerances
# [x] add in pandas dataframe charting infrastructure
# current array format: N_trial x M_sample x T_tol -> # frac paths rel opt

# [x] implement StateValidityChecker for d-dimensional snake environments
# [x] implement a ValidStateSampler so we do not need to rejection-sample in high-dimensional environments
# [x] work out math for strategic spots we need to check for full environment coverage in snake
# [x] implement tolerance verification algorithm
# [x] higher level utility function that does the check-advance-check-advance... pattern
# [x] Finder of points that are the connection radius away`
# [x] advance distance finder function in walls env
# [x] corner finder/decide on curve parameterization -> connection to current env representation algorithm for advance range finder
# [x] unit test for sanity (perhaps only the lower level bits, and then just to some quick cases with general PRMs in a low-D example)
# [x] save more data than just estimated success rate

# [ ] modularize the pieces of the script to be easily integrated into a supercloud mapreduce pattern
# [ ] design parameters and run experiment
# --what is likely going to happen is that given the current level of control we have form the python bindings, since we can only query individual paths, it's going to be too slow.
# --run anyway so we can present data to nick
# --but for interpretability, _now_ formally prove the result that means `if we want uniform converage with \epsilon-optimality' => we need an \epsilon'-net

# brief notes on how PRM does a solve call given a current roadmap, and some starts and goals
# solve():
# -- start states are added as milestones, then added to the graph in accordance to the connection rules
# -- goal states are added as milestones, then added to the graph in accordance to the connection rules\
# the roadmap is then built and solution is searched in two separate threads. we're interested in
#       checkForSolution() to see if we can run a small part of that process
# checkForSolution():
# -- goals are added again as milestones if there are any that have not been added to the graph yet.
# -- runs maybeConstructSolution()
# -- updates a pointer, loops... until terminated externally
# maybeConstructSolution():
# -- checks if the start/goal are in the same component, returns false with an empty PathPtr otherwise.
# -- checks if start/goal is pair allowed (certian pairs can be disallowed by the user), proceeds if so
# -- constructSolution with constructSolution()
# constructSolution():
# -- run boost astar_search on the underlying graph object.

# plan: it seems the plannerData is copies when passed in.
# so have original planner build the roadmap, clone the plannerData and then decouple it.
# stand up a new planner, pass in the plannerData.
# addMilestone the start and goal state.
# call construct path for the and then query the path information for the solution.





In [1]:
from nonasymptotic.envs import GrayCodeWalls
from nonasymptotic.prm import SimplePRM

import numpy as np
import pandas as pd

from datetime import datetime
import json
import os

env: OMPL_PATH=/home/seiji/Research/ompl/py-bindings


In [2]:
# defining free parameters: dimension, obstacles (i.e. state validity)
sample_schedule = [20]  #[10, 100, 1000, 10000, 100000]
tol = 0.5
delta = 0.25
n_trials = 2
ds = [2]  #[2, 6, 10, 14, 17, 20]
r = tol / np.sqrt(1 + tol * tol) * delta
test_seg_desc_num = 10  # should be a function of dimensionality in the future for coverage
n_nns = 10
machine_opener = 1e-2

# make a new log directory and save experiment params
name = 'long'
date_str = datetime.now().strftime('%m-%d-%Y')
log_dir = 'data/' + date_str + '_' + name
try:
    os.mkdir(log_dir)
except FileExistsError:
    print('WARNING: Made that log directory already!')

params = {
    "sample_schedule": sample_schedule,
    "tol": tol,
    "ds": ds,
    "n_trials": n_trials,
    "test_seg_desc_num": test_seg_desc_num,
    'machine_opener': machine_opener,
    'n_nns': n_nns
}

with open(os.path.join(log_dir, 'params.json'), 'w') as handle:
    json.dump(params, handle, indent=4)


In [ ]:
for d in ds:
    walls = GrayCodeWalls(d, 2, (1.0 - delta) / 2)

    fields = ['has_completeness', 'mean_stretch', 'stdev_stretch', 'n_nodes', 'n_edges', 'info']
    rec = {key: [] for key in fields}

    for i_trial in range(n_trials):
        prm = SimplePRM(r, walls.is_motion_valid, walls.sample_from_env, k_connection_neighbors=50)

        # save fraction of rel-optness
        for i_sample, m_samples in enumerate(sample_schedule):
            prm.grow_to_n_samples(m_samples)

            t = 0.0
            has_completeness = True
            len_diffs = []
            normed_r = r / walls.get_curve_arclength()
            while has_completeness and t < 1.0 - normed_r:
                print('t: %f' % t)
                # Step 1: find query points
                basepoint = walls.arclength_to_curve_point(t)
                testpoint = walls.arclength_to_curve_point(t + normed_r + machine_opener)  # march along

                # Step 2: run PRM algorithm on basepoint and test point. (we are going r ahead _on_ the provided path)
                sol_length, sol_path = prm.query_solution(basepoint, testpoint)

                # 1: compute the stretch difference
                # 2: find the point in the graph that the basepoint is connected to
                stretch_diff = (1 + tol) * r - sol_length
                # TODO: based on how we're querying the environment/scaling, r may not be the correct length of the path we are trying to approximate
                len_diffs += [stretch_diff]
                basepoint_nn = sol_path[0]
                d_first_edge = np.linalg.norm(basepoint_nn - basepoint)

                # Step 3: compute the advancement along the curve (but this is the point within the ball
                # that is within the slop radius of the nearest neighbor on the graph to the test point)
                # and yep! we need to also make sure that all the check points are in the same connected component.
                search_rad = min(d_first_edge + stretch_diff, r)
                for tick in range(test_seg_desc_num, -1):
                    advance = (normed_r / test_seg_desc_num) * tick
                    cand_point = walls.arclength_to_curve_point(t + advance)
                    if np.linalg.norm(cand_point - basepoint_nn) <= search_rad:
                        t += (advance + machine_opener)
                        new_testpoint = walls.arclength_to_curve_point(t)
                        new_start = ob.State(space)
                        for i in range(d):
                            new_start[i] = new_testpoint[i]
                        new_start_vertex = master_prm.addMilestone(new_start())
                        if not master_prm.sameComponent(new_start_vertex, start_vertex):
                            has_completeness = False
                            rec['info'] += ['graph covering path is not a single connected component']
                            # test_prm.clear()
                            break

                        # test_prm.clear()
                        continue

                has_completeness = False
                rec['info'] += ['could not find advance point']
                # test_prm.clear()
                break

            rec['has_completeness'] += [int(has_completeness)]
            rec['mean_stretch'] += [np.mean(len_diffs)]
            rec['stdev_stretch'] += [np.std(len_diffs)]
            rec['n_nodes'] += [prm.num_vertices()]
            rec['n_edges'] += [prm.num_edges()]

            # free the memory for next time. 
            prm.reset()

    # save data
    mi = pd.MultiIndex.from_product([range(n_trials), sample_schedule], names=['trial', 'm_samples'])
    df = pd.DataFrame(rec, index=mi)
    df.to_csv(os.path.join(log_dir, 'prm_%s_d%i.pkl' % (date_str, d)))



Info:    PRM: Space information setup was not yet called. Calling now.
t: 0.000000
